In [2]:
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# Given gridded time series data, calculates the spatially-averaged accumulation across
# drainage basins defined by 
def tempPlot():
    means1 = np.load('ensemble_mean_pr_sfc_Amon.npz', allow_pickle=True)
    accums = means1['xam']
    accums = np.multiply(accums, 31536000)
    basins = np.loadtxt('ant_full_drainagesystem_polygons.txt')
    print("hi")
    
    
    newNC = nc.Dataset('adaptor.mars.internal-1663556282.2356336-8299-12-f0261241-78f1-43ef-8b90-b2d6b9b5d118.nc')

    
    subsample = newNC.variables['tp'][:, 0:121:8, 0:1440:8]
    
    maskKey = subsample[0, 0, 0]
    for i in range(16):
        for j in range(180):
            if subsample[0, i, j] == maskKey:
                subsample[:, i, j] = np.nan
                
    predictYear = 0
                
    tempsAvg = np.mean(subsample[264:504], axis=0) 
    temps00 = np.mean(subsample[(predictYear*12-108):(predictYear*12-96)], axis=0)
    
    era5 = np.zeros((20, 16, 180))

    for i in range(20):
        era5[i] = np.mean(subsample[(i*12+264):(i*12+276)], axis=0) 

    era5 = np.subtract(era5, tempsAvg)
    era5 = np.flip(era5, axis=1)
    era5 = np.roll(era5, 90, axis=2)
    era5 = np.multiply(era5, 288000)
    
    
    trends = np.zeros((16, 180))
            
    basinAvgs = np.zeros(27)
    b = 1
    tempSum = 0
    count = 0
    
    
    basNo = 27
    yearSums = np.zeros(20)
    yearCount = np.zeros(20)
    rSums = np.zeros(28)
    ceSums = np.zeros(28)
    cellCount = 0.0
    
    areas = [0, 501448, 830000, 1565727, 247879, 190978, 610797, 493593, 159742, 146029, 919041, 257286, 722224, 1109771, 710953, 125183, 265243, 1852382, 269776, 382018, 201853, 217404, 214497, 91266, 160848, 34621, 43085, 54146]
    basinMaster = np.zeros((27, 20))
    
    for k in range(28):
        if k != 0:
            for i in basins:
                if i[2] == k:
                    lat = i[0]
                    lon = i[1]

                    if lon < 0:
                        lon = lon + 360

                    latInd = round(lat / 2) + 45
                    lonInd = round(lon / 2)

                    if lonInd == 180:
                        lonInd = 0

                    
                    if i[2] == b:
                        count = count + 1
                        tempSum = tempSum + trends[latInd, lonInd]
                    else:
                        b = b + 1
                        count = 1
                        tempSum = trends[latInd, lonInd]
                    

                
                    for j in range(20):
                        yearCount[j] = yearCount[j] + 1
                        yearSums[j] = yearSums[j] + accums[j, latInd, lonInd]

            timeSeries = np.divide(yearSums, yearCount)
            for i in range(20):
                basinMaster[k-1, i] = 0.9167*(timeSeries[i]*areas[k])/1000000
            print("________________________")
            yearSums = np.zeros(20)
            yearCount = np.zeros(20)
            b = 1
            tempSum = 0
            count = 0
            basinAvgs = np.zeros(27)
            
            
    AP = np.sum(basinMaster[23:], axis=0)
    for i in range(20):
        print(AP[i])
    print("________________________")
    WAIS = np.sum(basinMaster[17:23], axis=0) + basinMaster[0]
    for i in range(20):
        print(WAIS[i])
    print("________________________")
    EAIS = np.sum(basinMaster[1:17], axis=0) 
    for i in range(20):
        print(EAIS[i])
    print("________________________")
    AIS = AP + WAIS + EAIS
    for i in range(20):
        print(AIS[i])
    

    
           
    """
    ax = plt.axes(projection=ccrs.SouthPolarStereo())
    ax.set_extent([-180, 180, -90, -60], ccrs.PlateCarree())
    ax.gridlines()
    
    plt.rcParams['figure.figsize'] = [21, 21]
    
    plt.contourf(means1['lon'][0:16], means1['lat'][0:16], trends, levels=20, vmin=-50, vmax=50, transform=ccrs.PlateCarree(), cmap=plt.cm.seismic)
    plt.colorbar(orientation="horizontal", fraction=0.05, shrink=0.9, cmap=plt.cm.seismic)
    plt.title("1957-2000")
    
    ax.coastlines()
    
    plt.show()
    """
    
tempPlot()
    
    

hi
________________________
________________________
________________________
________________________
________________________
________________________
________________________
________________________
________________________
________________________
________________________
________________________
________________________
________________________
________________________
________________________
________________________
________________________
________________________
________________________
________________________
________________________
________________________
________________________
________________________
________________________
________________________
0.8768615039682774
0.24576547767272733
5.879022995128792
3.270064404739097
6.17266106508066
3.206919255618313
-0.4206276327480507
2.4527235172412833
14.52903799604653
-2.1960152421661143
0.9355282632514234
4.501943809319434
7.798317240222151
6.777564039537845
8.142240034695458
6.8948419209805785
-2.5046306520248547
4.831